In [ ]:
import ray

ray.init()

In [ ]:
import time


# A regular Python function.
def normal_function():
    return 1


# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def my_function():
    return 1


# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process.
obj_ref = my_function.remote()

# The result can be retrieved with ``ray.get``.
assert ray.get(obj_ref) == 1


@ray.remote
def slow_function(ii):
    time.sleep(1)
    print(ii)
    return ii


# Ray tasks are executed in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
# for ii in range(4):
#     # This doesn't block.
#     slow_function.remote(ii)
    
object_refs = [slow_function.remote(ii) for ii in range(1, 4)]
# Return as soon as one of the tasks finished execution.
ready_refs, remaining_refs = ray.wait(object_refs, num_returns=1, timeout=None)

In [ ]:
ready_refs

In [ ]:
remaining_refs

In [ ]:
ready_refs, remaining_refs = ray.wait(object_refs, timeout=None)

In [ ]:
ray.get(ready_refs)

In [ ]:
ray.get(remaining_refs)

In [2]:
# #@ray.remote
# class Counter(object):
        
#     def __init__(self, adder=None):
#         self.value = 0
        
#     def increment(self):
#         self.value += self.adder()
#         return self.value

#     def get_counter(self):
#         return self.value
    


# # Create an actor from this class.
# counter = Counter.remote()


# class RayServer(object):

#     def __getattribute__(self, attr):
#         return super(RayServer, self).__getattribute__(attr)

#     def __setattr__(self, attr, val):
#         return super(RayServer, self).__setattr__(attr, val)

#     def __getstate__(self):
#         return {}

#     def __setstate__(self, d):
#         pass
    
    
class Adder(object):
    
    value = 1
    
    def __init__(self):
        self.value = 1
        
    def __call__(self):
        return self.value
    
#     def __getattribute__(self, attr):
#         return super(Adder, self).__getattribute__(attr)
    
#     def __setattr__(self, attr, val):
#         return super(Adder, self).__setattr__(attr,val)
    
    
class Counter(object):
    
    value = 0
    
    def __init__(self, adder=None):
        if adder is None:
            self.adder = lambda: 1
        else:
            self.adder = adder
        self.value = 0

#     def __getattribute__(self, attr):
#         return super(Counter, self).__getattribute__(attr)
    
#     def __setattr__(self, attr, val):
#         return super(Counter, self).__setattr__(attr,val)
        
    def increment(self):
        self.value += self.adder()
        return self.value

    def get_counter(self):
        return self.value
    


In [ ]:
# # Call the actor.
# obj_ref = counter.increment.remote()
# print(ray.get(obj_ref))

In [ ]:
# # Create ten Counter actors.
# counters = [Counter.remote() for _ in range(10)]

# # Increment each Counter once and get the results. These tasks all happen in
# # parallel.
# results = ray.get([c.increment.remote() for c in counters])
# print(results)

# # Increment the first Counter five times. These tasks are executed serially
# # and share state.
# results = ray.get([counters[0].increment.remote() for _ in range(5)])
# print(results)

In [3]:
from tvb_multiscale.core.ray.client import create_ray_client, RayClient


In [4]:
adder = create_ray_client(Adder, 
                          ray_client_type=RayClient, 
                          non_blocking_methods=[])


In [5]:
adder.value

1

In [6]:
adder.ray_server._ray_method_signatures

{'__call__': [<Parameter "_ray_trace_ctx=None">],
 '__getattribute__': [<Parameter "attr">, <Parameter "_ray_trace_ctx=None">],
 '__getstate__': [<Parameter "_ray_trace_ctx=None">],
 '__init__': [<Parameter "_ray_trace_ctx=None">],
 '__ray_ready__': [<Parameter "_ray_trace_ctx=None">],
 '__ray_terminate__': [<Parameter "_ray_trace_ctx=None">],
 '__setattr__': [<Parameter "attr">,
  <Parameter "val">,
  <Parameter "_ray_trace_ctx=None">],
 '__setstate__': [<Parameter "d">, <Parameter "_ray_trace_ctx=None">]}

In [7]:
counter = create_ray_client(Counter, 
                            ray_client_type=RayClient, 
                            non_blocking_methods=["increment", "get_counter"], adder=adder)

In [8]:
counter.increment()

ObjectRef(32d950ec0ccf9d2a2b687f55c0e0e3c062f641bd0100000001000000)

In [9]:
counter.value

1

In [10]:
counter.adder.value = 2

In [11]:
counter.adder()

2

In [12]:
from tvb_multiscale.core.tvb.cosimulator.cosimulator import CoSimulator

In [13]:
remote_cosim = create_ray_client(CoSimulator, ray_client_type=RayClient)

In [14]:
remote_cosim.min_idelay_sync_n_step_ratio

1

In [15]:
remote_cosim.input_interfaces

In [16]:
ray.shutdown()